In [ ]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import squarify
import datetime as dt


In [ ]:
df = pd.read_excel('/Users/a18279786/Downloads/claims_sample_data.xlsx')

In [ ]:
df.info()

# Далее изучаю данные, произвожу нормализацию

In [ ]:
df.pivot_table(values = ['PAID_AMOUNT'],index = ['SERVICE_CATEGORY'],
               aggfunc= ['sum','count'])

In [ ]:
df.pivot_table(values = ['PAID_AMOUNT'],index = ['CLAIM_SPECIALTY'],
               aggfunc= ['sum','count']).reset_index().sort_values(by=('count', 'PAID_AMOUNT'),ascending=False)

In [ ]:
df.pivot_table(values = 'PAID_AMOUNT',index = 'PAYER',
               aggfunc= ['sum','count']).reset_index().sort_values(by=('count', 'PAID_AMOUNT'),ascending=False)

In [ ]:
df.pivot_table(values = ['PAID_AMOUNT'],index = ['MONTH'],
               aggfunc= ['sum','count']).head(5)

In [ ]:
df['MONTH'] = df['MONTH'].replace([201900],201901)
df['SERVICE_CATEGORY'] = df['SERVICE_CATEGORY'].replace(['SpecialistFFS'],'SpecialistsFFS')

In [ ]:
df.isnull().sum()

In [ ]:
df['CLAIM_SPECIALTY'].fillna('unknown',inplace = True)

In [ ]:
df.pivot_table(values = ['PAID_AMOUNT'],index = ['CLAIM_SPECIALTY','SERVICE_CATEGORY'],
               aggfunc= ['sum','count']).reset_index().sort_values(by=('count', 'PAID_AMOUNT'),ascending=False)

In [ ]:
df['MONTH'] = [str(i)[:4]+'-'+str(i)[4:6]+'-01' for i in df['MONTH']]
df['MONTH'].apply(pd.to_datetime)

In [ ]:
df.loc[df['PAID_AMOUNT']<0]

# Строю графики

In [ ]:
#treemap по платежам в разрезе SERVICE_CATEGORY и CLAIM_SPECIALTY
#
#

print('Введите дату начала в формате гггг-мм-дд:')
start = input()
print('Введите дату окончания в формате гггг-мм-дд:')
end = input()
a = df.loc[(df['MONTH']>=start) &(df['MONTH']<=end) & (df['PAID_AMOUNT']>0)] \
          .pivot_table(values = 'PAID_AMOUNT',index = ('SERVICE_CATEGORY','CLAIM_SPECIALTY'),
           aggfunc= 'sum').reset_index()



fig = px.treemap(a, path=[px.Constant('All'),'SERVICE_CATEGORY','CLAIM_SPECIALTY']
                    ,values='PAID_AMOUNT'
                    ,color = 'PAID_AMOUNT'
                    ,color_continuous_scale = 'rdylgn'
                    , width=1000, height=600)


fig.update_layout(
    #treemapcolorway = ('red'), #defines the colors in the treemap
    margin = dict(t=20, l=25, r=25, b=25) )
        
fig.show()

In [ ]:
# платежи по компаниям в тотал
b = df.loc[df['MONTH']>='2020-01-01'].pivot_table(values = 'PAID_AMOUNT',index = ('MONTH','PAYER'),
    aggfunc= 'sum').reset_index().sort_values(by=['MONTH','PAID_AMOUNT'], ascending=[True,False])

fig = px.bar(b, x='MONTH', y='PAID_AMOUNT',
             hover_data=['PAYER'], 
             color='PAYER', 
             height=500,
             title="Total pays"
            )

fig.update_layout(margin = dict(t=30, l=1, r=25, b=25) )
fig.show()

In [ ]:
#отрицательные суммы. Компании
с = df.loc[df['PAID_AMOUNT']<0].pivot_table(values = 'PAID_AMOUNT',index = ('MONTH','PAYER'),
    aggfunc= 'sum').reset_index().sort_values(by=['MONTH','PAID_AMOUNT'], ascending=[True,False])

fig = px.bar(с, x='MONTH', y='PAID_AMOUNT',
             hover_data=['PAYER'], 
             color='PAYER', 
             height=500,
             title="Negative sums by payers"
            )

fig.update_layout(margin = dict(t=30, l=1, r=25, b=25) )
fig.show()

In [ ]:
#отрицательные суммы. Категории
d = df.loc[df['PAID_AMOUNT']<0].pivot_table(values = 'PAID_AMOUNT',index = ('MONTH','SERVICE_CATEGORY'),
    aggfunc= 'sum').reset_index().sort_values(by=['MONTH','PAID_AMOUNT'], ascending=[True,False])

fig = px.bar(d, x='MONTH', y='PAID_AMOUNT',
             hover_data=['SERVICE_CATEGORY'], 
             color='SERVICE_CATEGORY', 
             height=500,
             title="Negative sums by category"
            )

fig.update_layout(margin = dict(t=30, l=1, r=25, b=25) )
fig.show()

In [ ]:
#ТОП-5 сервисов в каждой категории по сумме оплат за период

print('Введите дату начала в формате гггг-мм-дд:')
start = input()
print('Введите дату окончания в формате гггг-мм-дд:')
end = input()
e = df.loc[(df['MONTH']>=start) &(df['MONTH']<=end) & (df['PAID_AMOUNT']>0)] \
        .pivot_table(values = 'PAID_AMOUNT',index = ('SERVICE_CATEGORY','CLAIM_SPECIALTY'),
        aggfunc= 'sum').reset_index().sort_values(by=['SERVICE_CATEGORY','PAID_AMOUNT'], ascending=[True,False])
e['TOP'] = e.sort_values(['SERVICE_CATEGORY','PAID_AMOUNT'], ascending=[True,False]) \
             .groupby(['SERVICE_CATEGORY']) \
             .cumcount() + 1

e = e.loc[e['TOP']<=5]
e

In [ ]:
# Средний чек по месяцам по категориям

print('Введите дату начала в формате гггг-мм-дд:')
start = input()
print('Введите дату окончания в формате гггг-мм-дд:')
end = input()
f = df.loc[(df['MONTH']>=start) &(df['MONTH']<=end) & (df['PAID_AMOUNT']>0)] \
        .pivot_table(values = 'PAID_AMOUNT',index = ('MONTH','SERVICE_CATEGORY'),
        aggfunc= 'mean').reset_index().sort_values(by=['MONTH','PAID_AMOUNT'], ascending=[True,False])

fig = px.bar(f, x='MONTH', y='PAID_AMOUNT',
             hover_data=['SERVICE_CATEGORY'], 
             color='SERVICE_CATEGORY', 
             height=500,
             title="Mean bill by months, by category"
             )

fig.update_layout(margin = dict(t=30, l=1, r=25, b=25) )
fig.update_yaxes(visible=False, showticklabels=False)
fig.show()

In [ ]:
# Сравнение месяц к месяцу за предыдущие годы по платежам, по категориям

g = df.pivot_table(values = 'PAID_AMOUNT',index = ('MONTH','SERVICE_CATEGORY'),
        aggfunc= 'sum').reset_index().sort_values(by=['MONTH','PAID_AMOUNT'], ascending=[True,False])
for i in g.index:
    if int(str(g.loc[i,'MONTH']).split('-')[0]) < 2020:
        g.loc[i,'MM'] = '2020-'+str(g.loc[i,'MONTH']).split('-')[1]+'-'+str(g.loc[i,'MONTH']).split('-')[2]
    else:
        g.loc[i,'MM'] = g.loc[i,'MONTH']
        
    if int(str(g.loc[i,'MONTH']).split('-')[0]) == 2020:
        g.loc[i,'YY'] = 'TY'
    elif int(str(g.loc[i,'MONTH']).split('-')[0]) == 2019:
        g.loc[i,'YY'] = 'LY'
    else:
        g.loc[i,'YY'] = 'LLY'
g['MM'].apply(pd.to_datetime)   

print('Input category from list below: ')
print('')
for i in g['SERVICE_CATEGORY'].unique():
    print(i)

cat_name = input()

g1 = g.loc[(g['SERVICE_CATEGORY']==cat_name)&(g['MM']<='2020-07-01')]
fig = px.line(g1, x='MM', y='PAID_AMOUNT',
             hover_data=['YY'], 
             color='YY', 
             height=400,
             title="Month to month comparison: "+cat_name,
             markers=True 
             )

fig.update_layout(margin = dict(t=30, l=1, r=25, b=25),
                 legend_traceorder="reversed",
                  xaxis_title="Month",
                  yaxis_title="Paid amount",
                  legend_title="Year"
                 )


fig.show()